# Demand Prediction - Prophet

#### Creating a live demand forecasting model for a selected product and size combination
##### Note: The code must be run for the interactive plot to work (Use the attached sales_aggregated.csv file as the dataset)

In [1]:
# Uncomment the following lines if you need to install the required packages

# !pip install prophet
# !pip install ipywidgets
# !pip install plotly

In [2]:
# Required imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Loading the dataset
df = pd.read_csv('sales_aggregated.csv')
df

,Product,Size,Month,Sales
0,ALF Boys Ink Trousers,100,2022-01-01,6.0
1,ALF Boys Ink Trousers,100,2022-02-01,7.0
2,ALF Boys Ink Trousers,100,2022-03-01,1.0
3,ALF Boys Ink Trousers,100,2022-04-01,0.0
4,ALF Boys Ink Trousers,100,2022-05-01,1.0
...,...,...,...,...
19839,WHHS Snr Trouser,96,2025-01-01,6.0
19840,WHHS Snr Trouser,96,2025-02-01,5.0
19841,WHHS Snr Trouser,96,2025-03-01,2.0
19842,WHHS Snr Trouser,96,2025-04-01,0.0


### Prophet Implementation

##### Demand forecasting function

In [ ]:
# Function to forecast demand using Prophet

def forecast_demand(df, product, size, forecast_periods=6):
    # Ensure 'Month' is in datetime format
    df['Month'] = pd.to_datetime(df['Month'], errors='coerce')

    # Filter and prepare data
    df_subset = df[(df['Product'] == product) & (df['Size'] == size)][['Month', 'Sales']]
    df_subset = df_subset.rename(columns={'Month': 'ds', 'Sales': 'y'})

    # Build and fit the model
    model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    model.fit(df_subset)

    # Create future dataframe and generate forecast
    future = model.make_future_dataframe(periods=forecast_periods, freq='ME')
    forecast = model.predict(future)

    # Merge for evaluation
    forecast_full = forecast[['ds', 'yhat']].merge(df_subset, on='ds', how='left')
    forecast_train = forecast_full[forecast_full['y'].notnull()]

    # Evaluation metrics
    y_true, y_pred = forecast_train['y'], forecast_train['yhat']
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Display metrics
    print(f"Evaluation Metrics for {product} ({size}):")
    print(f"  R-squared: {r2:.4f}")
    print(f"  MSE:       {mse:.2f}")
    print(f"  RMSE:      {rmse:.2f}")
    print(f"  MAE:       {mae:.2f}")

    # Plot forecast and components
    model.plot(forecast)
    plt.title(f"Prophet Forecast - {product} ({size})")
    plt.show()

    model.plot_components(forecast)
    plt.show()

##### Funtion to create a plot for forecasted values only

In [ ]:
# Function to plot forecasted values only

def plot_forecasted_values_only(df, product, size, forecast_periods=6):
    df_subset = df[(df['Product'] == product) & (df['Size'] == size)][['Month', 'Sales']]
    df_subset = df_subset.rename(columns={'Month': 'ds', 'Sales': 'y'})

    model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    model.fit(df_subset)

    future = model.make_future_dataframe(periods=forecast_periods, freq='ME')
    forecast = model.predict(future)

    forecast_future = forecast.tail(forecast_periods)

    plt.figure(figsize=(8, 4))
    plt.plot(forecast_future['ds'], forecast_future['yhat'], marker='o', color='orange')
    plt.title(f"Forecasted Sales Only - {product} ({size})")
    plt.xlabel("Month")
    plt.ylabel("Forecasted Sales")
    plt.grid(True)
    plt.show()

##### Interactive Plot - Select the necessary product & size combination 
###### (The code should be run to use the interactive display)

In [6]:
# Display Plotly interactive widgets for product and size selection
product_dropdown = widgets.Dropdown(
    options=sorted(df['Product'].unique()),
    description="Product:"
)

size_dropdown = widgets.Dropdown(description="Size:")

def update_size_options(*args):
    selected_product = product_dropdown.value
    sizes = sorted(df[df['Product'] == selected_product]['Size'].unique())
    size_dropdown.options = sizes
    if sizes:
        size_dropdown.value = sizes[0]

product_dropdown.observe(update_size_options, names='value')
update_size_options()

display(product_dropdown, size_dropdown)

def update_forecast(product, size):
    forecast_demand(df, product, size)
    plot_forecasted_values_only(df, product, size)

widgets.interactive_output(update_forecast, {
    'product': product_dropdown,
    'size': size_dropdown
})

Dropdown(description='Product:', options=('ALF Boys Ink Trousers', 'ALF Jacket', 'ALF Jnr Blouse', 'ALF Jnr Sh…

Dropdown(description='Size:', options=('100', '104', '108', '112', '116', '120', '124', '128', '152', '64', '6…

Output()

---